In [1]:
!pip install sentence_transformers

In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 18.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# Load the abstracts dataset from a CSV file
# Assuming the CSV has a column named "abstract"
abstracts_df = pd.read_csv("/content/abstracts.csv")
abstracts = abstracts_df['Abstracts'].tolist()

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
# Initialize the embedding model
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

abstract_embeddings = embedder.encode(abstracts, convert_to_tensor=False)  # Set to False to get numpy array for FAISS
abstract_embeddings_np = np.array(abstract_embeddings)

# Build the FAISS index
index = faiss.IndexFlatL2(abstract_embeddings_np.shape[1])  # L2 distance is commonly used
index.add(abstract_embeddings_np)

print(f"Number of abstracts indexed: {index.ntotal}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Number of abstracts indexed: 719


In [5]:
def retrieve_abstracts(claim, k=1):
    """
    Retrieve the top-k relevant abstracts for a given claim.
    """
    claim_embedding = embedder.encode([claim], convert_to_tensor=False)  # Convert to numpy array
    claim_embedding_np = np.array(claim_embedding)

    # Search for the top-k most similar abstracts
    distances, indices = index.search(claim_embedding_np, k)
    retrieved_abstracts = [abstracts[i] for i in indices[0]]

    return retrieved_abstracts

# Example usage
claim = "They tell us that we are the primary forces controlling earth temperatures by the burning of fossil fuels and releasing their carbon dioxide."
retrieved_abstracts = retrieve_abstracts(claim)
print("Retrieved Abstracts:", retrieved_abstracts)


Retrieved Abstracts: ['Average temperature of the earth increases rapidly and climate also change accordingly. Main culprit behind it is the emission of greenhouse gas which is emitted from the burning of fossil fuels and also from other human activities. We can see in our daily lives how nature is changing. At the same time, in some areas are facing droughts and some other area have devastating floods killing many people within the same country. A recent report from NASA shows that a heat wave is blowing over major winter countries. As a result, the glaciers of the two polar provinces have started melting. Due to this climate change many organisms are disappearing from the earth. If it will continue in this way, one day the existence of all the creatures from our beloved earth will be disappeared. We are the only ones who can solve this problem through our behavioural and practical changes. In this study we highlighted the evil effects of global warming and climate change and also how

In [6]:
import pandas as pd

# Load the claims dataset from a CSV file, selecting only the relevant columns
df = pd.read_csv("/content/claims.csv", usecols=["claim", "claim_label"])
# Drop rows with missing claims or labels, if necessary
df = df.dropna(subset=['claim', 'claim_label'])

# Convert the 'label' column to integers
df['claim_label'] = df['claim_label'].astype(int)
# Rename "claim_label" to "label" for simplicity and consistency
df = df.rename(columns={"claim_label": "label"})

df['abstract'] = df['claim'].apply(lambda claim: " ".join(retrieve_abstracts(claim)))

In [7]:
df.to_csv("/content/ClaimWithRAG1.csv", index=False)

In [8]:
df.head()

claim  label  \
0  The sun has gone into  lockdown which could ca...      0   
1  Ironic' study finds more CO2 has slightly cool...      1   
2  They tell us that we are the primary forces co...      0   
3  If CO2 was so terrible for the planet, then in...      1   
4  Volcanoes Melting West Antarctic Glaciers, Not...      0   

                                            abstract  
0  According to NASA and IPCC, Global temperature...  
1  When William Shakespeare\ntook a breath, 280 m...  
2  Average temperature of the earth increases rap...  
3  In this chapter, the greenhouse effect, global...  
4  The global temperature rose by 0.2°C between t...